In [1]:
import numpy as np
import cv2
import matplotlib.pyplot as plt
import matplotlib.image as mpimg

from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import SGDClassifier
from sklearn.svm import LinearSVC
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.pipeline import FeatureUnion
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVC
from sklearn.linear_model import SGDClassifier

from moviepy.editor import VideoFileClip
from scipy.ndimage.measurements import label

from skimage.feature import hog
import glob
import time

%matplotlib inline

In [2]:
from util import *

In [3]:
# input dataset
cars = glob.glob('largeset/vehicles/*/*.png')+glob.glob('smallset/vehicles_smallset/*.jpeg')
notcars = glob.glob('largeset/non-vehicles/*/*.png')+glob.glob('./smallset/non-vehicles_smallset/*.jpeg')

In [4]:
# change the extract_features to a scikit learn compatible pipeline
class FeatureCreator(BaseEstimator, TransformerMixin):
    def __init__(self, color_space='RGB', spatial_size=(32, 32),
                        hist_bins=32, orient=11, 
                        pix_per_cell=16, cell_per_block=2, hog_channel='ALL',
                        spatial_feat=True, hist_feat=True, hog_feat=True):
        self.color_space  = color_space
        self.spatial_size = spatial_size
        self.hist_bins = hist_bins
        self.orient = orient
        self.pix_per_cell = pix_per_cell
        self.cell_per_block = cell_per_block
        self.hog_channel = hog_channel
        self.spatial_feat = spatial_feat
        self.hist_feat = hist_feat
        self.hog_feat = hog_feat
        
    def fit(self, x, y=None):
        return self
    
    def transform(self, X):
        return [single_img_features(single_x, self.color_space, self.spatial_size, self.hist_bins, 
                                   self.orient, self.pix_per_cell, 
                                   self.cell_per_block, self.hog_channel, self.spatial_feat, 
                                   self.hist_feat, self.hog_feat)
                for single_x in X]

In [5]:
car_imgs = [mpimg.imread(x) for x in cars]
notcar_imgs = [mpimg.imread(x) for x in notcars]

X = np.vstack((car_imgs, notcar_imgs)).astype(np.float64)
y = np.zeros(len(car_imgs)+len(notcar_imgs))
y[:len(car_imgs)] = 1

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.05, random_state=42)

In [ ]:
# Use a linear SVC 
model = Pipeline([
    ('feats', FeatureUnion([('create_feats', FeatureCreator())])), 
    ('scaler', StandardScaler()),
    ('linear svm', LinearSVC())])
model.fit(X_train, y_train)

Pipeline(steps=[('feats', FeatureUnion(n_jobs=1,
       transformer_list=[('create_feats', FeatureCreator(cell_per_block=2, color_space='RGB', hist_bins=32,
        hist_feat=True, hog_channel='ALL', hog_feat=True, orient=11,
        pix_per_cell=16, spatial_feat=True, spatial_size=(32, 32)))],
       transf...ax_iter=1000,
     multi_class='ovr', penalty='l2', random_state=None, tol=0.0001,
     verbose=0))])

In [ ]:
yhat_train = model.predict(X_train)
metric = accuracy_score(y_train, yhat_train)
print("Accuracy Train Rate: {}".format(metric))

In [ ]:
yhat_test = model.predict(X_test)
metric = accuracy_score(y_test, yhat_test)
print("Accuracy Test Rate: {}".format(metric))

Find cars in a box...

Goal: make a bunch of rectangles and return all rectangles which look like a car. 

In [ ]:
def search_windows_pipeline(img, windows, model):
    #1) Create an empty list to receive positive detection windows
    on_windows = []
    #2) Iterate over all windows in the list
    for window in windows:
        #3) Extract the test window from original image
        test_img = cv2.resize(img[window[0][1]:window[1][1], window[0][0]:window[1][0]], (64, 64))      
        # feed into pipeline
        prediction = model.predict([test_img])
        #7) If positive (prediction == 1) then save the window
        if prediction == 1:
            on_windows.append(window)
    #8) Return windows for positive detections
    return on_windows

Run a full pipeline!

In [ ]:
def process_image(image, model=model):
    # process everything based on the model pipeline used above
    windows = (slide_window(image, x_start_stop=[730, 1280], y_start_stop=[380, 550], 
                           xy_window=(96, 96), xy_overlap=(0.75, 0.75)))
    
    hot_windows = search_windows_pipeline(image, windows, model)  
    window_img = draw_boxes(image, hot_windows, color=(0, 0, 255), thick=6)                    
    
    # lets draw head boxes...
    heat = np.zeros_like(image[:,:,0])
    heat = add_heat(heat, hot_windows)    
    # Apply threshold to help remove false positives
    heat = apply_threshold(heat,1)
    # Visualize the heatmap when displaying    
    heatmap = np.clip(heat, 0, 255)

    labels = label(heatmap)
    draw_img = draw_labeled_bboxes(np.copy(image), labels)    
    return draw_img

In [ ]:
plt.imshow(process_image(image, model))

In [ ]:
# run for all images in test
test_images = glob.glob('./test_images/test*.jpg')

fig, axs = plt.subplots(3, 2, figsize=(16,14))
axs = axs.ravel()

for i, im in enumerate(test_images):
    axs[i].imshow(process_image(mpimg.imread(im)))

In [ ]:
#test_out_file = 'test_video_20170813.mp4'
#clip_test = VideoFileClip('test_video.mp4')
#clip_test_out = clip_test.fl_image(process_image)
#%time clip_test_out.write_videofile(test_out_file, audio=False)

In [ ]:
#test_out_file = 'project_video_20170813.mp4'
#clip_test = VideoFileClip('project_video.mp4')
#clip_test_out = clip_test.fl_image(process_image)
#%time clip_test_out.write_videofile(test_out_file, audio=False)